# Lab 1: Exploratory Data Analysis

by Nino Castellano, Will Lahners, Eddie Powers

### 1. Business Understanding

*** eddies part***

============================

[Dataset: Automobile Sales | Visual EDA](https://www.kaggle.com/code/ddosad/automobile-sales-visual-eda)

Can we identify any seasonal trends in toy sales, and how can this information be used for inventory planning? 

### 2. Data Understanding

##### 2.1 Data Description

In [7]:
#Required Libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from warnings import filterwarnings
filterwarnings('ignore')

In [13]:
#Loading the file
df = pd.read_csv("https://raw.githubusercontent.com/ncastellano2320/Machine-Learning-in-Python/86d705d3789e7cbfeb3ecabcbd34709c116b456c/Lab1_ExploringTableData/Auto%20Sales%20data.csv?token=GHSAT0AAAAAACNGPIZJTKC4KS2NBQD6INVAZN5MMQQ", parse_dates=['ORDERDATE'], dayfirst=True)

#Replace the deal size to represent small, medium, and large with "1", "2" or "3" respectively. 
df.DEALSIZE.replace(to_replace = ['Small', 'Medium', 'Large'],
                 value = range(1,4), inplace = True)

#Drop unnecessary data
df = df.drop(['PHONE', 'CUSTOMERNAME', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'ADDRESSLINE1', 'POSTALCODE', 'STATUS', 'ORDERLINENUMBER', 'DAYS_SINCE_LASTORDER', 'ORDERNUMBER', 'PRODUCTCODE'], axis=1)
df.head()

,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,PRODUCTLINE,MSRP,CITY,COUNTRY,DEALSIZE
0,30,95.70,2871.00,2018-02-24,Motorcycles,95,NYC,USA,1
1,34,81.35,2765.90,2018-05-07,Motorcycles,95,Reims,France,1
2,41,94.74,3884.34,2018-07-01,Motorcycles,95,Paris,France,2
3,45,83.26,3746.70,2018-08-25,Motorcycles,95,Pasadena,USA,2
4,36,96.66,3479.76,2018-10-28,Motorcycles,95,Burlingame,USA,2


Not only did we drop some unnecessary data, but we also changed the data type of the "DEALSIZE" variable, as it makes more sense to be processed as integers to represent small, medium, and large with "1", "2" or "3", respectively. 

In [9]:
#Find the data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ORDERNUMBER           2747 non-null   int64         
 1   QUANTITYORDERED       2747 non-null   int64         
 2   PRICEEACH             2747 non-null   float64       
 3   SALES                 2747 non-null   float64       
 4   ORDERDATE             2747 non-null   datetime64[ns]
 5   DAYS_SINCE_LASTORDER  2747 non-null   int64         
 6   PRODUCTLINE           2747 non-null   object        
 7   MSRP                  2747 non-null   int64         
 8   PRODUCTCODE           2747 non-null   object        
 9   CITY                  2747 non-null   object        
 10  COUNTRY               2747 non-null   object        
 11  DEALSIZE              2747 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 257.7+ K

Based on the above output of the dataframe, there are no missing values within our dataset as every column's non-null count adds up to the total number of rows in the data, 2747. 

Because there are no missing rows, it would be unnecessary to do any imputation on the data, as it is a complete dataset. 

In [10]:
#Describe the data
df.describe()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,DAYS_SINCE_LASTORDER,MSRP,DEALSIZE
count,2747.000000,2747.000000,2747.000000,2747.000000,2747,2747.000000,2747.000000,2747.000000
mean,10259.761558,35.103021,101.098952,3553.047583,2019-05-13 21:56:17.211503360,1757.085912,100.691664,1.601747
min,10100.000000,6.000000,26.880000,482.130000,2018-01-06 00:00:00,42.000000,33.000000,1.000000
25%,10181.000000,27.000000,68.745000,2204.350000,2018-11-08 00:00:00,1077.000000,68.000000,1.000000
50%,10264.000000,35.000000,95.550000,3184.800000,2019-06-24 00:00:00,1761.000000,99.000000,2.000000
75%,10334.500000,43.000000,127.100000,4503.095000,2019-11-17 00:00:00,2436.500000,124.000000,2.000000
max,10425.000000,97.000000,252.870000,14082.800000,2020-05-31 00:00:00,3562.000000,214.000000,3.000000
std,91.877521,9.762135,42.042549,1838.953901,NaN,819.280576,40.114802,0.591981


In [11]:
#More detailed descriptions of the data
data_des = pd.DataFrame()

data_des['Features'] = df.columns
data_des['Description'] = ['order number', 'how many items within the order',
                          'the price at which the items are sold',
                          'total revenue from the order', 'date the order was submitted',
                          'days since coustomer\'s last order', 'manufacturer\'s suggested retail price',
                          'size of the deal']
# data_des['Scales'] = ['nominal'] * 1 + ['ratio'] * 3 +['interval']
# data_des['Discrete\Continuous'] = ['continuous'] * 2 + ['discrete'] + \
#                                   ['continuous'] + ['discrete'] * 6
# data_des['Range'] = ['0.09 - 1.00', '0.36 - 1.00', '2 - 7', '96 - 310',
#                     '2 - 10'] + ['0: No; 1: Yes'] * 3 + [repr(set(df.department))] + \
#                     ['1: low; 2: medium; 3: high']

data_des


ValueError: Length of values (10) does not match length of index (12)